In [1]:
!pip install stable-baselines3[extra] mujoco gym
!pip install imageio
!pip install imageio-ffmpeg
!sudo apt-get update
!sudo apt-get install -y libosmesa6-dev libgl1-mesa-glx libglfw3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24

In [2]:
import os
import sys
import torch
import gym
from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.env_util import make_vec_env

# Detect if running in Colab/headless
def is_headless():
    return "google.colab" in sys.modules or not os.environ.get("DISPLAY")

# Set offscreen renderer
if is_headless():
    os.environ["MUJOCO_GL"] = "osmesa"

env_id = "Ant-v4"
train_env = make_vec_env(env_id, n_envs=1)

n_actions = train_env.action_space.shape[-1]
action_noise = NormalActionNoise(
    mean=torch.zeros(n_actions), sigma=0.1 * torch.ones(n_actions)
)

model = TD3(
    "MlpPolicy",
    train_env,
    action_noise=action_noise,
    verbose=1,
    batch_size=256,
    learning_rate=3e-4,
    buffer_size=10_000,
    train_freq=(1, "episode"),
    gradient_steps=-1,
    policy_kwargs=dict(net_arch=[400, 300])
)

model.learn(total_timesteps=10_000)
model.save("td3_ant_v4")
print("✅ Model trained and saved.")


/usr/local/lib/python3.11/dist-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment Ant-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


Using cuda device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 293      |
|    ep_rew_mean     | 164      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 121      |
|    time_elapsed    | 9        |
|    total_timesteps | 1171     |
| train/             |          |
|    actor_loss      | -2.32    |
|    critic_loss     | 0.0542   |
|    learning_rate   | 0.0003   |
|    n_updates       | 1000     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 526      |
|    ep_rew_mean     | -483     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 124      |
|    time_elapsed    | 33       |
|    total_timesteps | 4211     |
| train/             |          |
|    actor_loss      | -6.13    |
|    critic_loss     | 0.275    |
|    learning_rate   | 0.0003   |
|    n_updates       | 4076   